#Running SATIVA - positives

Prepare input files.
The alignment has been produced using ReproPhylo in the Sativa_eyorks_preandpost_protein notebook. We'll just cleanup the sequence headers and create a local copy.

In [1]:
cd eyorks/protein/development/

/home/working/Sativa/eyorks/protein/development


In [2]:
ls

931741477491194.75_rbcL@mafftLinsi_aln_clipped.fasta
AccessionBlacklist.txt*
align_eyorks.pkl
alignment.phy
eyorks_flora_curat1.gb
loci_counts.txt
loci.csv
nogene.txt
nr_eyorks.pkl
raw_eyorks.pkl
RAxML_info.931741477491194.75_rbcL@mafftLinsi_aln_clipped0
rbcL_cleanforSATIVA_eyorks.gb
rbcL_clustered_cleaned_eyorks.fa
rbcL_clustered_cleaned_eyorks.gb
rbcL_cropped_eyorks.fa
rbcL_cropped_eyorks.gb
rbcL_eyorks_id-1.uc
rbcL.log
rbcL@mafftLinsi_aln_clipped.phy
rbcL@mafftLinsi_aln.fasta
rbcL_nr_pre_Sativa_eyorks.gb
remove.txt
sativa/
species_vs_locus_raw.csv
target_locus_eyorks.csv
taxa.csv
tax_for_SATIVA.tax
taxids.txt


In [3]:
!cat rbcL@mafftLinsi_aln_clipped.phy | sed 's/_f[0-9] / /' > alignment.phy

SATIVA requires a 'taxonomy file', i.e. a text file that links each sequence record to a NCBI taxonomic ID (aka taxid).

We are going to generate this file from a bunch of sequence records downloaded from Genbank.

We have downloaded our reference sequences from Genbank in .gb format, which contains metadata including the taxid.

Let's parse the Genbank file.

In [4]:
from Bio import SeqIO

records = SeqIO.to_dict(SeqIO.parse(open('rbcL_nr_pre_Sativa_eyorks.gb','r'),'genbank'))

Before we proceed, there are three records I have manually identified that are still in this Genbank file but either will cause problems downstream, or are absent from the alignment. Let's remove them now.

In [5]:
!echo "AJ001005.1\nD44590.1\nD44561.1" > remove.txt

!cat remove.txt

AJ001005.1
D44590.1
D44561.1


In [6]:
from Bio import SeqIO

ids=[]
temp=[]
blacklist=[]
accessions = open('remove.txt','r')
dropped=0

for line in accessions:
    ids.append(line.strip())
    

recs_to_keep = {'rbcL': []}
recs_to_drop = {'rbcL': ids}

print "Blacklist recs to drop: %s" %recs_to_drop['rbcL']

for key in records.keys():
    r = records[key]
        
    if not r.id in recs_to_drop['rbcL']: 
        recs_to_keep['rbcL'].append(r.id)
        temp.append(r)
    
    else:
        print "exclude: %s" %r.id
        dropped+=1
        blacklist.append(r.id)
        
out = open('rbcL_cleanforSATIVA_eyorks.gb', 'w')
SeqIO.write(temp, out, 'genbank')
out.close()

print "Found blacklist records: %s (of %s)" %(dropped,len(recs_to_drop['rbcL']))
print "Kept: %s" %len(temp)

Blacklist recs to drop: ['AJ001005.1', 'D44590.1', 'D44561.1']
exclude: D44590.1
exclude: AJ001005.1
exclude: D44561.1
Found blacklist records: 3 (of 3)
Kept: 2399


Now re-import the cleaned Genbank file

In [7]:
from Bio import SeqIO

records = SeqIO.to_dict(SeqIO.parse(open('rbcL_cleanforSATIVA_eyorks.gb','r'),'genbank'))

Here is the first Genbank format record:

In [8]:
for r in records.keys():
    print records[r].format('genbank')
    break

LOCUS       AF206823                1377 bp    DNA     linear   PLN 26-JUL-2016
DEFINITION  Stellaria media ribulose 1,5-bisphosphate carboxylase (rbcL) gene,
            partial cds; chloroplast gene for chloroplast product.
ACCESSION   AF206823
VERSION     AF206823.1
KEYWORDS    .
SOURCE      chloroplast Stellaria media
  ORGANISM  Stellaria media
            Eukaryota; Viridiplantae; Streptophyta; Embryophyta; Tracheophyta;
            Spermatophyta; Magnoliophyta; eudicotyledons; Gunneridae;
            Pentapetalae; Caryophyllales; Caryophyllaceae; Alsineae; Stellaria.
REFERENCE   1  (bases 1 to 1377)
  AUTHORS   Soltis,P.S., Soltis,D.E. and Chase,M.W.
  TITLE     Angiosperm phylogeny inferred from multiple genes as a tool for
            comparative biology
  JOURNAL   Unpublished
REFERENCE   2  (bases 1 to 1377)
  AUTHORS   Soltis,P.S., Soltis,D.E. and Chase,M.W.
  TITLE     Direct Submission
  JOURNAL   Submitted (19-NOV-1999) School of Biological Sciences, Washington
         

Iterate over all records and check is taxid is subspecies, if yes, replace with taxid of species and extract taxonomy line. If it's a subspecies record the record ID for further processing.

In [9]:
from collections import defaultdict

taxon_to_taxid = {}
recs_to_adjust = []
taxon_to_recs = defaultdict(list)

for key in records.keys():
    r = records[key]
   
    source = [f for f in r.features if f.type == 'source'][0]
    if (len(source.qualifiers['organism'][0].split(" ")) == 2):
        print source.qualifiers['organism'][0],
        if 'db_xref' in source.qualifiers:
#            print source.qualifiers['db_xref']
            for t in source.qualifiers['db_xref']:
#                print t
                if 'taxon' in t:
                    if not source.qualifiers['organism'][0] in taxon_to_taxid:
                        print " .. add to records"
                        taxon_to_taxid[source.qualifiers['organism'][0]] = t.split(":")[1]
                    else:
                        print " .. already covered"
                    taxon_to_recs[source.qualifiers['organism'][0]].append(r.id)
    elif (len(source.qualifiers['organism'][0].split(" ")) > 2 and source.qualifiers['organism'][0].split(" ")[1] == 'sp.'):
        print source.qualifiers['organism'][0],
        if 'db_xref' in source.qualifiers:
            for t in source.qualifiers['db_xref']:
                if 'taxon' in t:
                    if not str(t) in taxon_to_taxid:
                        taxon_to_taxid[source.qualifiers['organism'][0]] = str(t)
                    else:
                        print " .. already covered" 
                    taxon_to_recs[source.qualifiers['organism'][0]].append(r.id)
    else:
        print "subspecies: %s" %source.qualifiers['organism'][0]
        recs_to_adjust.append(r.id)

Stellaria media  .. add to records
Brassica nigra  .. add to records
Neottia ovata  .. add to records
subspecies: Pinus sylvestris var. mongolica
Lilium pyrenaicum  .. add to records
Juncus tenuis  .. add to records
Brassica oleracea  .. add to records
Gnaphalium uliginosum  .. add to records
Acer pseudoplatanus  .. add to records
Abies alba  .. add to records
Rumex hydrolapathum  .. add to records
Sherardia arvensis  .. add to records
Persicaria capitata  .. add to records
subspecies: Dactylorhiza viridis var. viridis
Mercurialis annua  .. add to records
Scabiosa columbaria  .. add to records
Goodyera repens  .. add to records
Oxalis acetosella  .. add to records
Reynoutria japonica  .. add to records
Sagina maritima  .. add to records
Chaerophyllum temulum  .. add to records
Malva pusilla  .. add to records
Goodyera repens  .. already covered
Parapholis strigosa  .. add to records
Blitum bonus-henricus  .. add to records
Fallopia sachalinensis  .. add to records
Eleusine indica  .. a

For the records that were identified as being _subspecies_ reduce to species and check whether the taxid for the species had already been encountered. If not we'd need to fetch it from NCBI. I have identified one record which is not dealt with properly here, because it is a hybrid, so I have set up the script to manually remove this record, and two others which are in the Genbank file but not the alignment and so also need removing.

In [10]:
from collections import defaultdict

to_fetch=defaultdict(list)

for key in records.keys():
    r = records[key]
    if r.id in ['U06805.1']:
        source = [f for f in r.features if f.type == 'source'][0]
        adjust_from = source.qualifiers['organism'][0]
        print "%s: Manually removed hybrid" %(adjust_from)
    elif r.id in recs_to_adjust:
        source = [f for f in r.features if f.type == 'source'][0]
        adjust_from = source.qualifiers['organism'][0]
        adjust_to = " ".join(adjust_from.split(" ")[:2])
        print "%s -> %s" %(adjust_from,adjust_to)
        if adjust_to in taxon_to_taxid:
            taxon_to_recs[adjust_to].append(r.id)
        else:
            to_fetch[adjust_to].append(r.id)

Pinus sylvestris var. mongolica -> Pinus sylvestris
Dactylorhiza viridis var. viridis -> Dactylorhiza viridis
Amaranthus hybridus subsp. cruentus -> Amaranthus hybridus
Solidago canadensis var. canadensis -> Solidago canadensis
Equisetum hyemale subsp. affine -> Equisetum hyemale
Arrhenatherum elatius subsp. elatius -> Arrhenatherum elatius
Prunus domestica subsp. intermedia -> Prunus domestica
Solanum nigrum subsp. nigrum -> Solanum nigrum
Hedera helix subsp. rhizomatifera -> Hedera helix
Hieracium levicaule subsp. lepidulum -> Hieracium levicaule
Cyrtomium fortunei var. clivicola -> Cyrtomium fortunei
Picea abies var. abies -> Picea abies
Athyrium filix-femina var. angustum -> Athyrium filix-femina
Echinochloa crus-galli var. crus-galli -> Echinochloa crus-galli
Thelypteris palustris var. pubescens -> Thelypteris palustris
Solidago canadensis var. scabra -> Solidago canadensis
Silene vulgaris subsp. vulgaris -> Silene vulgaris
Asplenium trichomanes subsp. hastatum -> Asplenium tricho

Check if we are good or if any are missing.

In [11]:
if to_fetch:
    print "need to fetch some taxids"
else:
    print "Have taxids for all records"

need to fetch some taxids


In [12]:
print to_fetch

defaultdict(<type 'list'>, {'Primula veris': ['AF394976.1'], 'Bromus diandrus': ['HQ600435.1', 'HM849828.1', 'KF712964.1'], 'Dactylorhiza incarnata': ['KM360748.1'], 'Cotoneaster horizontalis': ['GU363807.1'], 'Odontites vernus': ['FJ395565.1'], 'Matricaria chamomilla': ['JN892268.1'], 'Hieracium levicaule': ['GQ120446.1'], 'Populus alba': ['KC485205.1'], 'Equisetum telmateia': ['AY226135.1'], 'Thymus praecox': ['KM361013.1', 'JN892956.1'], 'Brassica rapa': ['GQ184366.1', 'GQ184364.1', 'GQ184367.1', 'GQ184365.1']})


In [13]:
from Bio import Entrez
Entrez.email = "callumjmacgregor@gmail.com"

for binomial in to_fetch:
    print binomial
    handle = Entrez.esearch(db="Taxonomy", term=binomial)
    record = Entrez.read(handle)
    print record["IdList"][0]
    taxon_to_taxid[binomial] = record["IdList"][0]
    
    taxon_to_recs[binomial] = to_fetch[binomial]

Primula veris
170927
Bromus diandrus
37722
Dactylorhiza incarnata
230574
Cotoneaster horizontalis
690320
Odontites vernus
644195
Matricaria chamomilla
98504
Hieracium levicaule
651083
Populus alba
43335
Equisetum telmateia
3260
Thymus praecox
347388
Brassica rapa
3711


Write taxids to file and fetch full taxonomy for all of them using taxit from the taxtastic package.

In [14]:
taxids = []

out=open("taxids.txt",'w')
for sp in taxon_to_taxid:
    taxids.append(taxon_to_taxid[sp])
    out.write(taxon_to_taxid[sp]+"\n")
out.close()

Create tab-delimited text file with full taxonomic tree for each taxid.

In [15]:
!taxit taxtable -d /usr/bin/taxonomy.db -t taxids.txt -o taxa.csv

In order to make our lives easier downstream we will limit ourselves to only a defined set of the most common taxonomic levels, that should be known for pretty much all taxa: superkingdom, phylum, class, order, family, genus, species.

Extract 'taxonomy string' for a specific set of taxonomic levels.

In [16]:
from collections import defaultdict

tax_levels=['superkingdom','phylum','class','order','family','genus','species']
indices = []
taxdict = defaultdict(list)
taxids_to_taxonomy = {}

infile=open("taxa.csv",'r')
header=infile.next()

header_as_list=header.strip().replace('"','').split(",")
for i in range(len(header_as_list)):
#    print header_as_list[i]
    if header_as_list[i] in tax_levels:
#        print "\t"+header_as_list[i],i
        indices.append(i)

for line in infile:
    line_as_list=line.strip().replace('"',"").split(",")
    taxdict[line_as_list[0]] = line_as_list[1:]

infile.close()

for t in taxids:
    print t,
#    print taxdict[t]
    taxonomy=""
    for i in range(len(tax_levels)):
        if taxdict[t][indices[i]-1] == "":
            taxonomy+='unknown'+';'
        else:
            taxonomy+=taxdict[taxdict[t][indices[i]-1]][2]+";"
    print taxonomy[:-1]
    taxids_to_taxonomy[t] = taxonomy[:-1]


3752 Eukaryota;Streptophyta;unknown;Rosales;Rosaceae;Malus;Malus sylvestris
313104 Eukaryota;Streptophyta;unknown;Fabales;Fabaceae;Lathyrus;Lathyrus palustris
176248 Eukaryota;Streptophyta;unknown;Ericales;Ericaceae;Monotropa;Monotropa hypopitys
207839 Eukaryota;Streptophyta;unknown;Polypodiales;Dryopteridaceae;Cyrtomium;Cyrtomium fortunei
53809 Eukaryota;Streptophyta;unknown;Oxalidales;Oxalidaceae;Oxalis;Oxalis pes-caprae
262952 Eukaryota;Streptophyta;unknown;Polypodiales;Pteridaceae;Pteris;Pteris tremula
115624 Eukaryota;Streptophyta;unknown;Caryophyllales;Caryophyllaceae;Herniaria;Herniaria glabra
55378 Eukaryota;Streptophyta;unknown;Ericales;Balsaminaceae;Impatiens;Impatiens capensis
231488 Eukaryota;Streptophyta;Liliopsida;Liliales;Melanthiaceae;Paris;Paris quadrifolia
29760 Eukaryota;Streptophyta;unknown;Vitales;Vitaceae;Vitis;Vitis vinifera
234465 Eukaryota;Streptophyta;Liliopsida;Poales;Cyperaceae;Carex;Carex remota
39358 Eukaryota;Streptophyta;unknown;Lamiales;Lamiaceae;Prunel

Write out the *.tax file for SATIVA.

In [17]:
out=open("tax_for_SATIVA.tax", 'w')

for sp in taxon_to_recs:
    for rec in taxon_to_recs[sp]:
        out.write("%s\t%s\n" %(rec,taxids_to_taxonomy[taxon_to_taxid[sp]]))
        
out.close()

Check it

In [18]:
!head tax_for_SATIVA.tax

JQ391194.1	Eukaryota;Streptophyta;unknown;Rosales;Rosaceae;Malus;Malus sylvestris
JN890636.1	Eukaryota;Streptophyta;unknown;Fabales;Fabaceae;Lathyrus;Lathyrus palustris
KF997344.1	Eukaryota;Streptophyta;unknown;Ericales;Ericaceae;Monotropa;Monotropa hypopitys
EF394237.1	Eukaryota;Streptophyta;unknown;Polypodiales;Dryopteridaceae;Cyrtomium;Cyrtomium fortunei
KC795814.1	Eukaryota;Streptophyta;unknown;Polypodiales;Dryopteridaceae;Cyrtomium;Cyrtomium fortunei
KC795822.1	Eukaryota;Streptophyta;unknown;Polypodiales;Dryopteridaceae;Cyrtomium;Cyrtomium fortunei
AB575106.1	Eukaryota;Streptophyta;unknown;Polypodiales;Dryopteridaceae;Cyrtomium;Cyrtomium fortunei
AB598690.1	Eukaryota;Streptophyta;unknown;Polypodiales;Dryopteridaceae;Cyrtomium;Cyrtomium fortunei
AB598689.1	Eukaryota;Streptophyta;unknown;Polypodiales;Dryopteridaceae;Cyrtomium;Cyrtomium fortunei
AM235044.1	Eukaryota;Streptophyta;unknown;Oxalidales;Oxalidaceae;Oxalis;Oxalis pes-caprae


Let's now run SATIVA.

In [19]:
!./sativa/sativa.py -s alignment.phy -t tax_for_SATIVA.tax -x zoo -n rbcL -o ./ -T 5 -v


SATIVA v0.9-55-g0cbb090, released on 2016-06-28. Last version: https://github.com/amkozlov/sativa 
By A.Kozlov and J.Zhang, the Exelixis Lab. Based on RAxML 8.2.3 by A.Stamatakis.

SATIVA was called as follows:

./sativa/sativa.py -s alignment.phy -t tax_for_SATIVA.tax -x zoo -n rbcL -o ./ -T 5 -v

Mislabels search is running with the following parameters:
 Alignment:                        alignment.phy
 Taxonomy:                         tax_for_SATIVA.tax
 Output directory:                 /home/working/Sativa/eyorks/protein/development
 Job name / output files prefix:   rbcL
 Model of rate heterogeneity:      AUTO
 Confidence cut-off:               0.000000
 Number of threads:                5

*** STEP 1: Building the reference tree using provided alignment and taxonomic annotations ***

=> Loading taxonomy from file: tax_for_SATIVA.tax ...

==> Loading reference alignment from file: alignment.phy ...

Guessing input format: not fasta
Guessing input format: not phylip_relaxed
===>